In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [2]:
DF = pd.DataFrame()
url = "https://www.espn.co.uk/football/table/_/league/eng.1/season/{}"
for season in tqdm(range(2003, 2022)):
    teams = pd.read_html(url.format(season), attrs = {'class': 'Table Table--align-right Table--fixed Table--fixed-left'},  flavor='bs4', thousands ='.')[0]
    results = pd.read_html(url.format(season), attrs = {'class': 'Table Table--align-right'},  flavor='bs4', thousands ='.')[0]
    teams.columns = ['club']
    df = pd.merge(teams,results,on=teams.index,how='inner')
    df['SEASON'] = season
    DF = pd.concat([df, DF])

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:12<00:00,  3.81s/it]


In [3]:
DF.reset_index(inplace=True)

DF[['POSITION','CLUB_SF','CLUB']]=None
for i, r in DF.iterrows():
    name = r['club']
    position = re.findall(r'(\d*)',name)[0]
    club_short_form = name.replace(position,"")[:3]
    club_name = name.replace(position,"")[3:]
    DF['POSITION'].values[i] = position
    DF['CLUB_SF'].values[i] = club_short_form
    DF['CLUB'].values[i] = club_name
DF.drop(columns=['index', 'key_0', 'club'], inplace=True)

In [4]:
DF

,GP,W,D,L,F,A,GD,P,SEASON,POSITION,CLUB_SF,CLUB
0,31,23,5,3,72,20,52,74,2021,1,MNC,Manchester City
1,31,22,7,2,79,22,57,73,2021,2,LIV,Liverpool
2,30,18,8,4,64,23,41,62,2021,3,CHE,Chelsea
3,31,18,3,10,56,37,19,57,2021,4,TOT,Tottenham Hotspur
4,30,17,3,10,45,36,9,54,2021,5,ARS,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...
375,38,9,14,15,55,54,1,41,2003,16,MNC,Manchester City
376,38,9,12,17,45,57,-12,39,2003,17,EVE,Everton
377,38,6,15,17,48,65,-17,33,2003,18,LEI,Leicester City
378,38,8,9,21,40,79,-39,33,2003,19,LEE,Leeds United


In [5]:
DF.to_csv("Premier League Standings 2003-2021.csv", index=False, header=True)